## Word2vec model training
#### Model training based on three datasets' text data:
- M1: pwdb + eu_timeline  ( +  ireland_timeline )
- M2: ds_eu_cellar
- M3: M1+M2

#### Extract NOUN and NOUN PHRASES from each text data
#### Train the word2vec model with each dataset's textual data

### Import libraries

In [1]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))

import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import spacy
nlp = spacy.load('en_core_web_sm')

from typing import List, Tuple
import pandas as pd
from gensim.models import Word2Vec
from sem_covid.adapters.data_source import IndexTabularDataSource
from sem_covid.services.sc_wrangling.data_cleaning import (clean_text_from_specific_characters, clean_fix_unicode,
                                                           clean_remove_currency_symbols, clean_remove_emails,
                                                           clean_remove_urls)
from sem_covid.entrypoints.notebooks.language_modeling.language_model_tools.similarity_calculus import (
    euclidean_similarity, manhattan_similarity, cosine_similarity, get_similarity_matrix)
from sem_covid.entrypoints.notebooks.language_modeling.language_model_tools.document_handling_tools import \
    document_atomization_noun_phrases
from sem_covid.services.data_registry import Dataset

Forbidden: permission denied, on get http://srv.meaningfy.ws:8200/v1/mfy/data/air-flow

## Define constants

In [2]:
PWDB_TEXTUAL_CLASS = ['title', 'background_info_description', 'content_of_measure_description',
                      'use_of_measure_description', 'involvement_of_social_partners_description']

DEFAULT_TEXTUAL_COLUMN = ['title']
WINDOW = 5
MIN_COUNT = 1
VECTOR_SIZE = 300
EPOCHS = 50
EU_TIMELINE_TOTAL_EXAMPLES = 171
IRELAND_TIMELINE_TOTAL_EXAMPLES = 410
EU_CELLAR_TOTAL_EXAMPLES = 2653

## Data preprocessing
- data cleanup
- turn corpus into spacy document

In [28]:
def apply_cleaning_functions(document_corpus: pd.Series) -> pd.Series:
    """
    This function receives the document and leads through cleaning steps
    Args:
        document_corpus: dataset document corpus

    Returns: clean document corpus
    """
    unused_characters = ["\\r", ">", "\n", "\\", "<", "''", "%", "...", "\'", '"', "(", "\n", "*", "1)", "2)", "3)",
                         "[", "]", "-", "_", "\r"]

    new_document_corpus = document_corpus.apply(clean_text_from_specific_characters, characters=unused_characters)
    new_document_corpus = new_document_corpus.apply(clean_fix_unicode)
    new_document_corpus = new_document_corpus.apply(clean_remove_urls)
    new_document_corpus = new_document_corpus.apply(clean_remove_emails)
    new_document_corpus = new_document_corpus.apply(clean_remove_currency_symbols)

    return new_document_corpus

In [4]:
class LanguageModelPipeline:
    """
        This pipeline executes the steps for word2vec language training.
    """

    def __init__(self, dataset_sources: List[Tuple[IndexTabularDataSource, List[str]]]):
        """
            :param dataset_sources: represents the source of the datasets.
        """
        self.dataset_sources = dataset_sources
        self.documents_corpus = pd.Series()
        self.word2vec = None

    def download_datasets(self):
        """
            In this step it will download the dataset and detect selected columns.
            It can be downloaded as many datasets as there are in data source.
        """
        self.dataset_sources = [(dataset_columns, dataset_source.fetch())
                                for dataset_source, dataset_columns in self.dataset_sources]

    def extract_textual_data(self):
        """
            After downloading the datasets, the textual data will be found and and concatenated
            with executing of several steps as well. It will fill the NaN values with empty space,
            add a dot at the end of each concatenated column and reset the index.
        """
        self.documents_corpus = pd.concat([dataset[columns]
                                          .fillna(value="")
                                          .agg('. '.join, axis=1)
                                          .reset_index(drop=True)
                                           for columns, dataset in self.dataset_sources
                                           ], ignore_index=True)

    def clean_textual_data(self):
        """
            The next step is data cleaning. In this step the function "apply_cleaning_functions"
            applies the following actions:
                - clean the document from specific characters
                - delete unicode
                - removes emails and URLs and currency symbols
        """
        self.documents_corpus = apply_cleaning_functions(self.documents_corpus)

    def transform_to_spacy_doc(self):
        """
            When the document is clean, is going to be transform into spacy document
        """
        self.documents_corpus = self.documents_corpus.apply(nlp)

    def extract_features(self):
        """
            To extract the parts of speech, below it was defined classes for each token is necessary.
        """
        self.documents_corpus = pd.concat([self.documents_corpus,
                                           self.documents_corpus.apply(document_atomization_noun_phrases)]
                                          , ignore_index=True)
        self.documents_corpus = self.documents_corpus.apply(lambda x: list(map(str, x)))

    def model_training(self):
        """
            When the data is prepared it's stored into Word2Vec model.
        """
        self.word2vec = Word2Vec(sentences=self.documents_corpus, window=WINDOW,
                                 min_count=MIN_COUNT, vector_size=VECTOR_SIZE)

    def execute(self):
        """
            The final step is execution, where are stored each step and it will be executed in a row
        """
        self.download_datasets()
        self.extract_textual_data()
        self.clean_textual_data()
        self.transform_to_spacy_doc()
        self.extract_features()
        self.model_training()

## Experiment Nr#1 language model based on:
- PWDB
- eu-timeline
- ireland-timeline

In [24]:
dataset_sources_config = [
    (Dataset.PWDB, PWDB_TEXTUAL_CLASS),
    (Dataset.EU_ACTION_TIMELINE, DEFAULT_TEXTUAL_COLUMN),
    (Dataset.IRELAND_ACTION_TIMELINE, DEFAULT_TEXTUAL_COLUMN)
]
model1_language_model_pipeline = LanguageModelPipeline(dataset_sources=dataset_sources_config)
model1_language_model_pipeline.execute()

## Experiment Nr#2 language model based on:
- eu-cellar


In [25]:
dataset_sources_config = [
    (Dataset.EU_CELLAR, DEFAULT_TEXTUAL_COLUMN)
]
model2_language_model_pipeline = LanguageModelPipeline(dataset_sources=dataset_sources_config)
model2_language_model_pipeline.execute()

## Experiment Nr#3 language model based on:
- PWDB
- eu-timeline
- ireland-timeline
- eu-cellar

In [26]:
dataset_sources_config = [
    (Dataset.PWDB, PWDB_TEXTUAL_CLASS),
    (Dataset.EU_ACTION_TIMELINE, DEFAULT_TEXTUAL_COLUMN),
    (Dataset.IRELAND_ACTION_TIMELINE, DEFAULT_TEXTUAL_COLUMN),
    (Dataset.EU_CELLAR, DEFAULT_TEXTUAL_COLUMN)
]
model3_language_model_pipeline = LanguageModelPipeline(dataset_sources=dataset_sources_config)
model3_language_model_pipeline.execute()

## Similarity matrices
### Euclidean similarity

In [2]:
%%time
model1_euclidean_similarity_matrix = get_similarity_matrix(wv=model1_language_model_pipeline.word2vec.wv,
                                                           similarity_function=euclidean_similarity)
model1_euclidean_similarity_matrix

NameError: name 'get_similarity_matrix' is not defined

In [ ]:
%%time
model2_euclidean_similarity_matrix = get_similarity_matrix(wv=model2_language_model_pipeline.word2vec.wv,
                                                           similarity_function=euclidean_similarity)
model2_euclidean_similarity_matrix

In [ ]:
%%time
model3_euclidean_similarity_matrix = get_similarity_matrix(wv=model3_language_model_pipeline.word2vec.wv,
                                                           similarity_function=euclidean_similarity)
model3_euclidean_similarity_matrix

### Cosine similarity

In [ ]:
%%time
model1_cosine_similarity_matrix = get_similarity_matrix(wv=model1_language_model_pipeline.word2vec.wv,
                                                           similarity_function=cosine_similarity)
model1_cosine_similarity_matrix

In [ ]:
%%time
model2_cosine_similarity_matrix = get_similarity_matrix(wv=model2_language_model_pipeline.word2vec.wv,
                                                           similarity_function=cosine_similarity)
model2_cosine_similarity_matrix

In [ ]:
%%time
model3_cosine_similarity_matrix = get_similarity_matrix(wv=model3_language_model_pipeline.word2vec.wv,
                                                           similarity_function=cosine_similarity)
model3_cosine_similarity_matrix

### Manhattan similarity

In [ ]:
%%time
model1_manhattan_similarity_matrix = get_similarity_matrix(wv=model1_language_model_pipeline.word2vec.wv,
                                                           similarity_function=manhattan_similarity)
model1_manhattan_similarity_matrix

In [ ]:
%%time
model2_manhattan_similarity_matrix = get_similarity_matrix(wv=model2_language_model_pipeline.word2vec.wv,
                                                           similarity_function=manhattan_similarity)
model2_manhattan_similarity_matrix

In [ ]:
%%time
model3_manhattan_similarity_matrix = get_similarity_matrix(wv=model3_language_model_pipeline.word2vec.wv,
                                                           similarity_function=manhattan_similarity)
model3_manhattan_similarity_matrix